In [3]:
%reset -f

from functools import reduce
import sys
import time
import math
import numpy as np
import pandas as pd
from itertools import combinations_with_replacement
from datetime import datetime

from stats import Stats
from character import Character

# File path and name
file_path = "res.txt"
SLOT_NUMER = 37

#============getting all the items from the db.csv file=========================#
print(f'Fetching items from db: Started')
start_time = time.time()

all_items       = []
all_items_ids   = { k:[] for k in range(SLOT_NUMER)}

df = pd.read_csv('db.csv')
for index, row in df.iterrows():
    d = row.dropna().to_dict()
    
    all_items_ids[d['slot']].append(d['id'])
    all_items.append(Stats(d['slot'], d['name'], d['set'],attributes=d))
print(len(all_items), all_items)
#=======filter items_in_slot and prepare for the combination function===========#
all_items_ids_filtered = list(filter(None, all_items_ids.values()))

#================print some info about the objects loaded=======================#
for slot, item in all_items_ids.items():
    if len(item) > 0:
        print(f'slot: {slot:3}, choices:{len(item):3}')
all_items_ids_filtered_len = reduce(lambda a, b: a*len(b), [1,*all_items_ids_filtered])
print(f'Items combinations: {all_items_ids_filtered_len}')

print(f'Fetching items from db: Completed ({time.time()-start_time}s)')
print(f'Size all_items_ids: {sys.getsizeof(all_items_ids)}')
print(f'Size all_items: {sys.getsizeof(all_items)}')

Fetching items from db: Started
36 [<stats.Stats object at 0x0000017BB1123BD0>, <stats.Stats object at 0x0000017BB1122090>, <stats.Stats object at 0x0000017BB1123C90>, <stats.Stats object at 0x0000017BB1122A10>, <stats.Stats object at 0x0000017BB1123B50>, <stats.Stats object at 0x0000017BB11220D0>, <stats.Stats object at 0x0000017BB1144090>, <stats.Stats object at 0x0000017BB1122050>, <stats.Stats object at 0x0000017BB1144050>, <stats.Stats object at 0x0000017BB1122150>, <stats.Stats object at 0x0000017BB1123D90>, <stats.Stats object at 0x0000017BB11448D0>, <stats.Stats object at 0x0000017BB1144A50>, <stats.Stats object at 0x0000017BB1144810>, <stats.Stats object at 0x0000017BB1144790>, <stats.Stats object at 0x0000017BB11449D0>, <stats.Stats object at 0x0000017BB1144890>, <stats.Stats object at 0x0000017BB1144C10>, <stats.Stats object at 0x0000017BB1144C90>, <stats.Stats object at 0x0000017BB1144D10>, <stats.Stats object at 0x0000017BB1144D90>, <stats.Stats object at 0x0000017BB1144E1

In [5]:
#==============base stats for a specific class (not present in db)==============#
base_class_stats    = { 'phy_atk':1202,	'mag_atk':1202,	'phy_def':301,	'mag_def':301,	'hp':157000,
                        'dmg_per':      18,  
                        'dmg_per_m':    0,
                        'crit':         25,        
                        'maxi':         0,        
                        'crit_dmg':     25,

                        'pola':         0,
                        'specific':     0,
                        'all_s_dmg':    3,
                        'boss_dmg':     0,
                        'adapt':        0,
                        'cdr':          0
                       }

#===============================force passives==================================#
head_hunter = {'dmg_per':-24, 'boss_dmg':80}


#============================sockets combinations===============================#
avail_sockets = [  
                    Stats('0', 'socket', attributes={'maxi':6}),
                    Stats('0', 'socket', attributes={'crit':6}),
                    Stats('0', 'socket', attributes={'boss_dmg':2.5})
                ]

print(f'Generating sockets combinations: Started')
start_time = time.time()
# generate combinations
sockets_combinations = list(combinations_with_replacement(avail_sockets, 38))

sockets_combinations_len = len(sockets_combinations)
print(f'Socket combinations: {sockets_combinations_len}')

# generate item for each socket combination, grouping sockets into a single item
itemized_sockets = []
for sockets in sockets_combinations:
    tot_socket = Stats('0', 'SOCKETS')
    for socket in sockets:
        tot_socket += socket
    itemized_sockets.append(tot_socket)

print(len(itemized_sockets))
itemized_sockets = list(filter(Stats.maxed_crit, itemized_sockets))
print(len(itemized_sockets))
itemized_sockets = list(filter(Stats.maxed_maxi, itemized_sockets))
print(len(itemized_sockets))

itemized_sockets_ids = np.arange(len(itemized_sockets))

print(f'Generating sockets combinations: Completed ({time.time()-start_time}s)')
print(f'Size itemized_sockets: {sys.getsizeof(itemized_sockets)}')

Generating sockets combinations: Started
Socket combinations: 780
780
660
582
Generating sockets combinations: Completed (0.10899543762207031s)
Size itemized_sockets: 4792


In [6]:
#=========================combinatory for all items=============================#
print('Generating Items combinations, Starting')
start_time = time.time()

combs = np.array(np.meshgrid(*all_items_ids_filtered, itemized_sockets_ids))
combs = combs.T.reshape(-1,len([*all_items_ids_filtered, itemized_sockets_ids]))
print(combs)

#combs = list(product(*list(items_in_slot_filtered), itemized_sockets))

#combs = random.sample(combs, 100000)
#combs = combs[::500]

combs_len = len(combs)
print(f'Total combinations: {combs_len}')

print(f'Total combinations as expected: {combs_len==(all_items_ids_filtered_len*sockets_combinations_len)}')
print(f'Combinations, Complete ({time.time()-start_time}s)')

print(f'Size combs: {sys.getsizeof(combs)}')



Generating Items combinations, Starting
[[ 21   0   1 ...  25  35   0]
 [ 21   4   1 ...  25  35   0]
 [ 21   8   1 ...  25  35   0]
 ...
 [ 31   4  13 ...  34  35 581]
 [ 31   8  13 ...  34  35 581]
 [ 31  12  13 ...  34  35 581]]
Total combinations: 57212928
Total combinations as expected: False
Combinations, Complete (37.5115532875061s)
Size combs: 128


In [ ]:
#=============================create specific items ============================#
base_class_stats_item = Stats('base','class base stats','',attributes=base_class_stats)
head_hunter_force_passive = Stats('HH','unique HH','',attributes=head_hunter)

new_character = Character('Char', '')
new_character = new_character.equip(base_class_stats_item, base_class_stats_item.slot)
new_character = new_character.equip(head_hunter_force_passive, head_hunter_force_passive.slot)

In [9]:
import multiprocessing

def process_task(combs_part):
    
    print('Item placement and damage calculation, Starting')
    
    print(f"{'':24} / {combs_len}",end='\r')

    inv_combs_len = round(1/combs_len,3)

    zeroed = 0

    mul_list = []
    for i, items in enumerate(combs):
        # create a new "character" for each item combination
        new_character.name = 'jeff_%s'%i
        new_character.items = {}
        new_character.attributes = {}

        # add the sockets item with is a Stats object
        
        new_character = new_character.equip(itemized_sockets[items[-1]], itemized_sockets[items[-1]].slot)
        
        # equip all itemsD
        for id in items[0:-1]:
            new_character = new_character.equip(all_items[id], all_items[id].slot)
        
        # save to list
        mul_list.append(new_character.get_mul())

        zeroed += int(mul_list[-1] == 0)
        
        print(f"[{zeroed*inv_combs_len:5}]{i*inv_combs_len:5}({sys.getsizeof(mul_list)})", end='\r')
    
    print(f'\nItem placement and damage calculation, Complete ({time.time()-start_time}s)')
    print(f'Size mul_list: {sys.getsizeof(mul_list)}')

if __name__ == "__main__":
    num_tasks = 4
    with multiprocessing.Pool() as pool:
        pool.map(process_task, np.array_split(combs, num_tasks))

AttributeError: module '__main__' has no attribute '__spec__'

In [7]:


#===========equip items combinations and specific items if needed===============#
print('Item placement and damage calculation, Starting')
start_time = time.time()

print(f"{'':24} / {combs_len}",end='\r')

inv_combs_len = round(1/combs_len,3)

zeroed = 0
mul_list = []
for i, items in enumerate(combs):
    # create a new "character" for each item combination
    new_character.name = 'jeff_%s'%i
    new_character.items = {}
    new_character.attributes = {}

    # add the sockets item with is a Stats object
    
    new_character = new_character.equip(itemized_sockets[items[-1]], itemized_sockets[items[-1]].slot)
    
    # equip all itemsD
    for id in items[0:-1]:
        new_character = new_character.equip(all_items[id], all_items[id].slot)
    
    # save to list
    mul_list.append(new_character.get_mul())

    zeroed += int(mul_list[-1] == 0)
    
    print(f"[{zeroed*inv_combs_len:5}]{i*inv_combs_len:5}({sys.getsizeof(mul_list)})", end='\r')

print(f'\nItem placement and damage calculation, Complete ({time.time()-start_time}s)')
print(f'Size mul_list: {sys.getsizeof(mul_list)}')

Item placement and damage calculation, Starting


KeyboardInterrupt: 

In [ ]:
#==============apply the damage calculation to each character===================#

best_comb_index = mul_list.index(max(mul_list))
best_comb = combs[best_comb_index]

best_character = Character('Char', '')
best_character = new_character.equip(base_class_stats_item, base_class_stats_item.slot)
best_character = new_character.equip(head_hunter_force_passive, head_hunter_force_passive.slot)

best_character = best_character.equip(best_comb[-1], best_comb[-1].slot)

for id in best_comb[:-1]:
    best_character = best_character.equip(all_items[id], all_items[id].slot)

best = new_character.get_damage()






In [ ]:
#=========================save to file best N entries===========================#
with open(file_path, "w") as file:
    file.write(f'Latest update: {datetime.now()}\n')
    
    output_string = f"{math.trunc(best[0]):<10}{math.trunc(best[1]):<10}\n{best[2]}\n"
    
    file.write(output_string.replace('|0.0','|   '))
    
print('DONE')

In [12]:
import multiprocessing

def square(number):
    return number * number

if __name__ == "__main__":
    # Data
    numbers = [1, 2, 3, 4, 5]

    # Create a Pool of workers
    num_workers = multiprocessing.cpu_count()  # Number of CPU cores available
    pool = multiprocessing.Pool(processes=num_workers)

    # Use the map function to apply the square function to each element in parallel
    results = pool.map(square, numbers)

    # Close the pool to release resources
    pool.close()
    pool.join()

    # Print the results
    print("Original numbers:", numbers)
    print("Squared numbers:", results)


AttributeError: module '__main__' has no attribute '__spec__'